# Spaceship Titanic - Seventh Model

## Overview

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, 
the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets 
orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic
collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000
years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!

In this competition your task is to predict whether a passenger was transported to an alternate dimension during the 
Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal
records recovered from the ship's damaged computer system.

## File and Data Field Descriptions

### train.csv 

Personal records for about two-thirds (~8700) of the passengers, to be used as training data.

| Column Name | Description |
|------------- |-------------|
| `PassengerId` | A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always. |
| `HomePlanet` | The planet the passenger departed from, typically their planet of permanent residence. |
| `CryoSleep` | Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins. |
| `Cabin` | The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard. |
| `Destination` | The planet the passenger will be debarking to. |
| `Age` | The age of the passenger. |
| `VIP` | Whether the passenger has paid for special VIP service during the voyage. |
| `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` | Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities. |
| `Name` | The first and last names of the passenger. |
| `Transported` | Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict. |

### test.csv

Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. 

Your task is to predict the value of Transported for the passengers in this set.
    
### sample_submission.csv

A sample submission file in the correct format.

| Column Name | Description |
|------------- |-------------|
| `PassengerId` | A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always. |
| `Transported` | Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict. |

In [64]:
model='kbinopt'

In [65]:
# Library imports

# Data wrangling
import pandas as pd
import numpy as np
import missingno
from collections import Counter

# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Machine learning models
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# Model evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# Data preprocessing

## Load Data

In [66]:
# Load the data - to pandas dataframes

test_df = pd.read_csv('./inputs/test.csv')
test_idx = test_df['PassengerId']
train_df = pd.read_csv('./inputs/train.csv')

## Drop unrequired features & add engineered ones

In [67]:
# split name into components
def split_name(name:str):
    if pd.isnull(name): # protect against empty values
        return (np.nan, np.nan, np.nan)
    parts = name.split(' ')
    first = parts[0]
    last = parts[-1]
    return (first, last)

# split cabin into components
def split_cabin(name:str):
    if pd.isnull(name): # protect against empty values
        return (np.nan, np.nan, np.nan)
    parts = name.split('/')
    deck = parts[0]
    side = parts[-1]
    number = ' '.join(parts[1:-1])
    return (deck, number, side)

# This function will drop and add required features
def feature_manipulation(d : pd.DataFrame) -> pd.DataFrame:
    # print the shape of the provided dataframe
    print("Before: ", d.shape)
    # Clone the provided dataframe
    df = d.copy()

    _, df['Name'] = zip(*df['Name'].map(split_name))
    
    # Location
    df['Deck'], _, df['Side'] = zip(*df['Cabin'].map(split_cabin))
    df['DeckSide'] = df['Deck'] + df['Side'] # Combine Deck and Side

    # Group related columns
    df['Group'] = df['PassengerId'].map(lambda x: x.split('_')[0])
    df['GroupSize'] = df['Group'].map(df['Group'].value_counts())
    
    print("After: ", df.shape)
    return df

In [68]:
train_df = feature_manipulation(train_df)

Before:  (8693, 14)
After:  (8693, 19)


In [69]:
test_df = feature_manipulation(test_df)

Before:  (4277, 13)
After:  (4277, 18)


In [70]:
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Side,DeckSide,Group,GroupSize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Ofracculy,False,B,P,BP,0001,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Vines,True,F,S,FS,0002,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Susent,False,A,S,AS,0003,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Susent,False,A,S,AS,0003,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Santantines,True,F,S,FS,0004,1


In [71]:
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Side,DeckSide,Group,GroupSize
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Carsoning,G,S,GS,0013,1
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Peckers,F,S,FS,0018,1
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Unhearfus,C,S,CS,0019,1
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Caltilter,C,S,CS,0021,1
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Harperez,F,S,FS,0023,1


## Handle Missing Data

In the first model we had a complicated approach - here we will just use the median for numeric and mode for non-numeric

In [72]:
# Define a function to print missing data
def print_missing(train: pd.DataFrame, test: pd.DataFrame):
    missing_data = pd.DataFrame({
        'Train Missing': train.isnull().sum(),
        'Test Missing': test.isnull().sum()
    }).sort_values(by='Train Missing', ascending=False)

    print(missing_data)

In [73]:
# Check for missing data before imputing
print_missing(train_df, test_df)

              Train Missing  Test Missing
CryoSleep               217          93.0
ShoppingMall            208          98.0
VIP                     203          93.0
HomePlanet              201          87.0
Name                    200          94.0
Cabin                   199         100.0
Deck                    199         100.0
DeckSide                199         100.0
Side                    199         100.0
VRDeck                  188          80.0
FoodCourt               183         106.0
Spa                     183         101.0
Destination             182          92.0
RoomService             181          82.0
Age                     179          91.0
GroupSize                 0           0.0
PassengerId               0           0.0
Group                     0           0.0
Transported               0           NaN


### Define functions to replace data

In [74]:
def fill_with_median(df, features):
    for feature in features:
        df[feature] = df[feature].fillna(df[feature].median())
    return df

def fill_with_mode(df, features):
    for feature in features:
        df[feature] = df[feature].fillna(df[feature].mode()[0])
    return df

def fill_with_mean(df, features):
    for feature in features:
        df[feature] = df[feature].fillna(df[feature].mean())
    return df

# Here we provide a list of other_features - if a row has a nan in a feature we will fill it with the mean of the other_features in its row 
def fill_with_mean_of_other_features(df, features, other_features):
    for feature in features:
        df[feature] = df[feature].fillna(df[other_features].dropna().mean(axis=1))
    return df

def fill_with_constant(df, features, constant):
    for feature in features:
        df[feature] = df[feature].fillna(constant)
    return df

### Apply simple rules for empty data

In [75]:
def handle_empty_data(df: pd.DataFrame) -> pd.DataFrame:
   
    # fill in missing values with a constant
    df = fill_with_constant(df, ['VIP'], False)
    df = fill_with_constant(df, ['Name', 'Cabin', 'Deck', 'Side', 'DeckSide', 'HomePlanet', 'Destination'], 'Unknown')

    # Fill numerics with median
    df = fill_with_median(df, ['Age'])
    
    # Fill with average of other spend features
    spend_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    df = fill_with_mean_of_other_features(df, spend_features, spend_features)
    df = fill_with_constant(df, spend_features, 0)

    # Have a sensible default so leave this as is
    # for the provided dataframe set CryoSleep to False if TotalSpend is greater than 0
    UnknownCryoSpender = (df["CryoSleep"].isnull() | df["CryoSleep"].isna()) & (df[spend_features].dropna().sum(axis=1) > 0)
    df.loc[UnknownCryoSpender, 'CryoSleep'] = False
    df['CryoSleep'] = df['CryoSleep'].fillna('Unknown')

    return df

In [76]:
train_df = handle_empty_data(train_df)
test_df = handle_empty_data(test_df)

In [77]:
# Check for missing data after imputing
print_missing(train_df, test_df)

              Train Missing  Test Missing
Age                       0           0.0
Name                      0           0.0
VIP                       0           0.0
Transported               0           NaN
Spa                       0           0.0
Side                      0           0.0
ShoppingMall              0           0.0
RoomService               0           0.0
PassengerId               0           0.0
HomePlanet                0           0.0
Cabin                     0           0.0
GroupSize                 0           0.0
Group                     0           0.0
FoodCourt                 0           0.0
Destination               0           0.0
DeckSide                  0           0.0
Deck                      0           0.0
CryoSleep                 0           0.0
VRDeck                    0           0.0


In [78]:
# Look at the training data after the changes to remove empty data
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Side,DeckSide,Group,GroupSize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Ofracculy,False,B,P,BP,0001,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Vines,True,F,S,FS,0002,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Susent,False,A,S,AS,0003,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Susent,False,A,S,AS,0003,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Santantines,True,F,S,FS,0004,1


In [79]:
# Look at the test data
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Side,DeckSide,Group,GroupSize
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Carsoning,G,S,GS,0013,1
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Peckers,F,S,FS,0018,1
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Unhearfus,C,S,CS,0019,1
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Caltilter,C,S,CS,0021,1
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Harperez,F,S,FS,0023,1


In [80]:
# Check the shapes:

print(f'Train: {train_df.shape}')
print(f'Test: {test_df.shape}')

# Check the train and test column data types side by side
pd.concat([train_df.dtypes, test_df.dtypes], axis=1, keys=['Train', 'Test'])

Train: (8693, 19)
Test: (4277, 18)


,Train,Test
PassengerId,object,object
HomePlanet,object,object
CryoSleep,object,object
Cabin,object,object
Destination,object,object
Age,float64,float64
VIP,bool,bool
RoomService,float64,float64
FoodCourt,float64,float64
ShoppingMall,float64,float64


## Drop Unrequired columns

We can now drop these columns:

- Cabin
- TotalSpend
- PassengerId - may do this one last so we have the key

And to simplify for now lets also remove
- Name
- Group

In [81]:
# Drop the columns that are not required
def drop_unrequired_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns_to_drop = [
        'Cabin',
        'Name',
        'DeckSide'
    ]
    df.drop(columns_to_drop, axis = 1, inplace = True)
    return df

train_df = drop_unrequired_columns(train_df)
test_df = drop_unrequired_columns(test_df)

## Prepare the data for training

- Log/Bin the numeric data
- OneHotEncoding

In [82]:
# There are some cases where we have used 'Unknown' in boolean columns that have True and False otherwise that led to an error
# complaining about a mix of str and bool so we need to convert the columns to str

# print all columns where the value is 'Unknown'
for column in train_df.columns:
    if 'Unknown' in train_df[column].unique():
        print(column)
        train_df[column] = train_df[column].astype(str)

HomePlanet
CryoSleep
Destination
Deck
Side


## Run the pipeline

In [83]:
# what data looks like before pipeline
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side,Group,GroupSize
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,P,0001,1
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,S,0002,1
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,S,0003,2
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,S,0003,2
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,S,0004,1


In [84]:
# Define the preprocessor
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer

# Define pipelines for the features where we wish to bin and OHE
def preprocessor(n_bins: int, strategy='quantile', scaling_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], categorical_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']):
    scaling_pipeline = Pipeline([
        ('scale', RobustScaler()),
    ])

    age_pipeline = Pipeline([
        ('scale', RobustScaler())
    ])
    if n_bins > 0:
        age_pipeline = Pipeline([
            ('binning', KBinsDiscretizer(n_bins=n_bins, encode='onehot-dense', strategy=strategy))
        ])

    categorical_pipeline = Pipeline([
        ('onehot', OneHotEncoder())
    ])
    return ColumnTransformer(
        transformers=[
            ('category', categorical_pipeline, categorical_features),
            ('spending', scaling_pipeline, scaling_features),
            ('age', age_pipeline, ['Age'])
        ],
        remainder='passthrough'  # This leaves the rest of the columns untouched
    )

In [85]:
y = train_df.pop('Transported')

In [86]:
i = train_df.pop('PassengerId')

# Time to attempt the modelling

## Split the data to create a validation set

In [87]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_df, y, test_size = 0.25, random_state = 0)

## Instantiation of the models

In [88]:
# Function to score models
def score_model(classifier, X_train, X_val, y_train, y_val):
    # Fit the model
    classifier.fit(X_train, y_train)

    # Predict the test data
    y_pred = classifier.predict(X_val)

    # Create a confusion matrix
    cm = confusion_matrix(y_val, y_pred)

    # Calculate the accuracy
    acc = accuracy_score(y_val, y_pred)

    scores = cross_val_score(classifier, X_train, y_train, cv = 10)

    return { 
            'confusion_matrix': cm,
            'accuracy': acc,
            'kfold-cv': scores.mean()
        }

def run_classifiers(classifiers: dict, X_train, X_test, y_train, y_val) -> dict:
    results = {}
    for name, classifier in classifiers.items():
        results[name] = score_model(classifier, X_train, X_test, y_train, y_val)
    return results

In [89]:
# Model definitions
classifiers = {
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
}

## Preprocess the split data & Train the models

In [91]:
results = {}
bins_to_try = [0]
for x in range(2, 20):
    bins_to_try.append(x)
for n_bins in bins_to_try:
    prep = preprocessor(n_bins)
    results[f'quan_{n_bins}'] = list(run_classifiers(classifiers, prep.fit_transform(X_train), prep.transform(X_val), y_train, y_val).values())[0]
    prep = preprocessor(n_bins, strategy='uniform')
    results[f'uni_{n_bins}'] = list(run_classifiers(classifiers, prep.fit_transform(X_train), prep.transform(X_val), y_train, y_val).values())[0]
    prep = preprocessor(n_bins, strategy='kmeans')
    results[f'km_{n_bins}'] = list(run_classifiers(classifiers, prep.fit_transform(X_train), prep.transform(X_val), y_train, y_val).values())[0]

In [92]:
# convert results to a dataframe
results_df = pd.DataFrame(results).T
results_df.sort_values(by='kfold-cv', ascending=False)

,confusion_matrix,accuracy,kfold-cv
uni_8,"[[855, 222], [215, 882]]",0.798988,0.804109
uni_11,"[[861, 216], [218, 879]]",0.800368,0.803955
km_18,"[[866, 211], [220, 877]]",0.801748,0.802423
quan_6,"[[853, 224], [211, 886]]",0.799908,0.802269
km_11,"[[859, 218], [204, 893]]",0.805888,0.801962
uni_18,"[[863, 214], [210, 887]]",0.804968,0.801962
km_16,"[[851, 226], [205, 892]]",0.801748,0.801809
uni_14,"[[869, 208], [204, 893]]",0.810488,0.801347
km_10,"[[864, 213], [204, 893]]",0.808188,0.801195
km_12,"[[853, 224], [221, 876]]",0.795308,0.801194


In [93]:
# best mean of kfold-cv plus accuracy
results_df['kfold-cv'].max()

0.8041092985779311

## Tuning the two best models

## Tuning the best models

In [94]:
# Tune a model using GridSearchCV
def tune_model(classifier, param_grid):
    print(f'\n\nTuning {classifier.__class__.__name__}...')
    grid_search = GridSearchCV(classifier, param_grid, cv=10, n_jobs = -1, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_estimator = grid_search.best_estimator_
    score = score_model(best_estimator, X_train, X_val, y_train, y_val)

    print(f'Best SVC Accuracy: { score["accuracy"] }')
    print(f'Best SVC KFold CV: { score["kfold-cv"] }')
    f1 = f1_score(y_val, best_estimator.predict(X_val))
    # and finally the f1 score
    print(f'F1 Score: {f1}')

    return (grid_search.best_params_, best_estimator, f1)

In [95]:
tuning_results = [
    tune_model(
        XGBClassifier(),
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'learning_rate': [0.1, 0.01, 0.001],
            'eval_metric': ['logloss'], 
            'use_label_encoder': [False]
        }, 
    ),
    tune_model(
        RandomForestClassifier(), 
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'criterion': ['gini', 'entropy']
        }
    ),
    tune_model(
        CatBoostClassifier(verbose=0),
        param_grid = {
            'iterations': [100, 200, 300],
            'depth': [3, 4, 5],
            'learning_rate': [0.1, 0.01, 0.001],
        }
    ),
    tune_model(
        LGBMClassifier(),
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'learning_rate': [0.1, 0.01, 0.001],
        }
    ),
]



Tuning XGBClassifier...


ValueError: 
All the 270 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/n18576/source/edoatley/ml/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/n18576/source/edoatley/ml/.venv/lib/python3.9/site-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/Users/n18576/source/edoatley/ml/.venv/lib/python3.9/site-packages/xgboost/sklearn.py", line 1500, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "/Users/n18576/source/edoatley/ml/.venv/lib/python3.9/site-packages/xgboost/sklearn.py", line 521, in _wrap_evaluation_matrices
    trai

In [ ]:
# Print out the F1 Scores for the different models
for result in tuning_results:
    print(f'F1 Score for {result[1].__class__}: {result[2]}')

In [ ]:
best_result = max(tuning_results, key=lambda x: x[2])[1]

## Create submission

Now we can predict the test set and submit

In [ ]:
# Convert any Boolean columns with Unknown to String type
for column in test_df.columns:
    if 'Unknown' in test_df[column].unique():
        print(column)
        test_df[column] = test_df[column].astype(str)

In [ ]:
# Check the shape

print(f'Test: {test_df.shape}')

# Print the first 5 rows:
test_df.head()

In [ ]:
# Capture the passenger IDs for submission and remove from the data as not useful predictors
idx_test = test_df.pop('PassengerId')

In [ ]:
# Pre-process test set
X_test = preprocessor.transform(test_df) # sparse to dense array

In [ ]:
# Predict the test set
y_pred = best_result.predict(X_test).astype(bool)

In [ ]:
# Write the submission DataFrame
submission = pd.DataFrame({'PassengerId': idx_test, 'Transported': y_pred})
submission.to_csv(f'submissions/{model}-submission.csv', index=False)
print("Your submission was successfully saved!")
